This blog post is an extended guide on instruction-tuning Llama 2 from Meta AI. The idea of the blog post is to focus on creating the instruction dataset, which we can then use to fine-tune the base model of Llama 2 to follow our instructions. 

The goal is to create a model which can create instructions based on input. The idea behind this is that this can then be used for others to create instruction data from inputs. That's especially helpful if you want to personalize models for, e.g., tweeting, email writing, etc, which means that you would be able to generate an instruction dataset from your emails to then train a model to mimic your email writing. 

Okay, so can we get started on this? In the blog, we are going to:

1. Define our use case in detail and create a prompt template for our instructions
2. Create an instruction dataset
3. Instruction-tune Llama 2 using `trl` and the `SFTTrainer` 
4. Test the Model and run Inference

## 1. Define our use case in detail and create a template for our instructions

Before we describe our use case, we need to better understand what even is an instruction. 

> An instruction is a piece of text or prompt that is provided to an LLM, like Llama, GPT-4, or Claude, to guide it to generate a response. Instructions allow humans to steer the conversation and constrain the language model's output to be more natural, useful, and aligned with the user's goals. Crafting clear, well-formulated instructions is key to productive conversations.
> 

Examples of instructions are listed below in the table.

| Capability | Example Instruction |
| --- | --- |
| Brainstorming | Provide a diverse set of creative ideas for new flavors of ice cream. |
| Classification | Categorize these movies as either comedy, drama, or horror based on the plot summary. |
| Closed QA | Answer the question 'What is the capital of France?' with a single word. |
| Generation | Write a poem in the style of Robert Frost about nature and the changing seasons. |
| Information Extraction | Extract the names of the main characters from this short story. |
| Open QA | Why do leaves change color in autumn? Explain the scientific reasons. |
| Summarization | Summarize this article on recent advancements in renewable energy in 2-3 sentences. |

As described in the beginning, we want to fine-tune a model to be able to generate instructions based on input. (output). We want to use this as a way to create synthetic datasets to personalize LLMs and Agents. 

Converting the idea into a basic prompt template following the [Alpaca format](https://github.com/tatsu-lab/stanford_alpaca#data-release) we get. 

```python
### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

### Input:
Dear [boss name],

I'm writing to request next week, August 1st through August 4th,
off as paid time off.

I have some personal matters to attend to that week that require 
me to be out of the office. I wanted to give you as much advance 
notice as possible so you can plan accordingly while I am away.

Please let me know if you need any additional information from me 
or have any concerns with me taking next week off. I appreciate you 
considering this request.

Thank you, [Your name]

### Response:
Write an email to my boss that I need next week 08/01 - 08/04 off.
```

## 2. Create an instruction dataset

After we defined our use case and prompt template, we need to create our instruction dataset. Creating a high-quality instruction dataset is key for a good-performing model. Research shows that [“Less Is More for Alignment”](https://arxiv.org/abs/2305.11206) shows that creating a high-quality, low-quantity (~1000 samples) dataset can achieve the same performance as less-quality and high-quantity datasets. 

There are several ways to create an instruction dataset, including: 

1. Using an existing dataset and converting it into an instruction dataset, e.g., [FLAN](https://huggingface.co/datasets/SirNeural/flan_v2)
2. Use existing LLMs to create synthetically instruction datasets, e.g., [Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca)
3. Use Humans to create instructions datasets, e.g., [Dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k). 

Each of the methods has its own advantages and disadvantages and depends on the budget, time, and quality requirements. For example, using an existing dataset is the easiest but might not be tailored to your specific use case, while using humans might be the most accurate but can be time-consuming and expensive. It is also possible to combine several methods to create an instruction dataset, as shown in [Orca: Progressive Learning from Complex Explanation Traces of GPT-4.](https://arxiv.org/abs/2306.02707)

To keep it simple, we are going to use **[Dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k)** an open-source dataset of instruction-following records generated by thousands of Databricks employees in several of the behavioral categories outlined in the **[InstructGPT paper](https://arxiv.org/abs/2203.02155)**, including brainstorming, classification, closed QA, generation, information extraction, open QA, and summarization.

Let's start coding, but first, let's install our dependencies.

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

To load the **`databricks/databricks-dolly-15k`** dataset, we use the **`load_dataset()`** method from the 🤗 Datasets library.

In [1]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# dataset size: 15011

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011
{'instruction': 'Can you tell me something about Stanley J. Goldberg', 'context': 'Stanley J. Goldberg (born Maryland, 1939) is a special trial judge of the United States Tax Court.\n\nGoldberg attended public schools in Baltimore, MD. He earned a B.S. from the University of Maryland, School of Business and Public Administration in 1960 and an LL.B. from the University of Maryland School of Law in 1964. He did graduate work in Federal Income Taxation at New York University. Goldberg was admitted to practice in Maryland and New Jersey, 1964 and 1967, and Federal District Court. He began his career as a Tax Attorney in January 1965 with the United States Department of Treasury, Office of Chief Counsel, Internal Revenue Service, in New York City and was initially assigned to the General Litigation function. In 1967, he was reassigned to the Tax Litigation function. In 1976, he was promoted to Special Trial Attorney, and then to Assistant District Counsel in 1984. He wa

To instruct tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a **`formatting_function`** that takes a sample and returns a string with our format instruction.

In [11]:
def format_instruction(sample):
	return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

### Input:
{sample['response']}

### Response:
{sample['instruction']}
"""

# def format_instruction(sample):
#   output_texts = []
#   for instruction, response in zip(sample['instruction'], sample['response']):
#     output_texts.append(f"""### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

# ### Input:
# {response}

# ### Response:
# {instruction}""")
#   return output_texts

Let's test our formatting function on a random example.

In [3]:
from random import randrange

s = randrange(len(dataset))
print(format_instruction(dataset[s:s+1])[0])

### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

### Input:
The president of the United States (POTUS) is the head of state and head of government of the United States of America and the commander-in-chief of the United States Armed Forces. The president is also the head of the executive branch of the federal government of the United States and is the chairman of the presidential cabinet.

Joe Biden is the 46th and current president of the United States, in office since January 2021.

### Response:
Who is the 46th president of the United States?


## 3. Instruction-tune Llama 2 using `trl` and the `SFTTrainer`

 We will use the recently introduced method in the paper "[QLoRA: Quantization-aware Low-Rank Adapter Tuning for Language Generation](https://arxiv.org/abs/2305.14314)" by Tim Dettmers et al. QLoRA is a new technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance. The TL;DR; of how QLoRA works is:

- Quantize the pre-trained model to 4 bits and freeze it.
- Attach small, trainable adapter layers. (LoRA)
- Finetune only the adapter layers while using the frozen quantized model for context.

If you want to learn more about QLoRA and how it works, I recommend you to read the [Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA](https://huggingface.co/blog/4bit-transformers-bitsandbytes) blog post.

First, we are going to load our model together with our quantization configuration.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "NousResearch/Llama-2-7b-hf"

# BitsAndBytesConfig int-4 config 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
model.config.pretraining_tp = 1 

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `SFTTrainer`  supports a native integration with `peft`, which makes it super easy to efficiently instruction tune LLMs. We only need to create our `LoRAConfig` and provide it to the trainer.

In [5]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM", 
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


Before we can start our training we need to define the hyperparameters (`TrainingArguments`) we want to use.

In [2]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="llama-7-int4-dolly",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

We now have every building block we need to create our `SFTTrainer` to start then training our model.

In [12]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction, 
    args=args,
)

Start training our model by calling the `train()` method on our `Trainer` instance.

In [13]:
# train
trainer.train()

Step,Training Loss
10,1.451600
20,1.344500
30,1.303900
40,1.232000
50,1.272200
60,1.278800
70,1.232000
80,1.247100
90,1.255200
100,1.263000


TrainOutput(global_step=858, training_loss=1.1997463252994565, metrics={'train_runtime': 11675.6292, 'train_samples_per_second': 3.857, 'train_steps_per_second': 0.964, 'total_flos': 1.4350831872638976e+17, 'train_loss': 1.1997463252994565, 'epoch': 2.08})

The training took 04:36:00 on a `g5.2xlarge`. The instance costs `1,212$/h` which brings us to a total cost of `54$`. 

## 4. Test Model and run Inference

After the training is done we want to run and test our model. We will use `peft` and `transformers` to load our LoRA adapter into our model. We will also use `accelerate` to run our inference on multiple GPUs. 

In [6]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

args.output_dir = "llama-7-int4-dolly/checkpoint-858"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
) 
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Let’s load the dataset again with a random sample to try the summarization.

In [28]:
from datasets import load_dataset 
from random import randrange


# Load dataset from the hub and get a sample
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

### Input:
{sample['response']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['instruction']}")

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Prompt:
New Jersey, New York, Ohio, Oklahoma - USA
Kolkata, New Delhi - India

Generated instruction:
Classify the above places based on whether they are in India or USA

Ground truth:
Classify the below places based on the country they belong to.

New Jersey, New York, Kolkata, New Delhi, Ohio, Oklahoma


Nice! our model works! If want to accelerate our model we can deploy it with [Text Generation Inference](https://github.com/huggingface/text-generation-inference). Therefore we would need to merge our adapter weights into the base model.

In [30]:
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
) 

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]